In [ ]:
import os
import sys
import json

curdir = os.getcwd()
while 'config.json' not in os.listdir(curdir):
    curdir = os.path.dirname(curdir)
with open(os.path.join(curdir,'config.json'), 'r') as f:
    dataset = json.load(f)['dataset']

curdir = os.getcwd()
while 'filepathhelper.py' not in os.listdir(curdir):
       curdir = os.path.dirname(curdir)
sys.path.append(curdir)
import filepathhelper

In [ ]:
import os
import pickle
import json
import pandas as pd
import numpy as np
# from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm

In [ ]:
winissues = pd.read_csv(filepathhelper.path(dataset,'winissue.csv'))
assignees = pd.read_csv(filepathhelper.path(dataset,'assignee.csv'),sep=';')
# assignees = assignees[(assignees['issuekey'].isin(winissues['issuekey']))]
teams = pd.read_csv(filepathhelper.path(dataset,'team.csv'))
# filterteamsize = pd.read_csv(filepathhelper.path(dataset,'filterteamsize.csv'))


closeresolve = pd.read_csv(filepathhelper.path(dataset,'closeresolve.csv'),sep=';')
teams = teams[(teams['issuekey'].isin(closeresolve['issuekey']))]
teams = teams[(teams['issuekey'].isin(assignees['issuekey']))]
# teams = teams[(teams['issuekey'].isin(filterteamsize['issuekey']))]
assignees = assignees[(assignees['issuekey'].isin(teams['issuekey']))]
descript = pd.read_csv(filepathhelper.path(dataset,'description.csv'))
descript.set_index('issuekey',inplace=True)

In [ ]:
info = pd.read_csv(filepathhelper.path(dataset,'issueinformation.csv'),sep=';')
# proj = set()
# temp = info['issuekey'].str.split('-')
# for i in temp:
#     proj.add(i[0])
# print(len(proj))
info = info[(info['issuekey'].isin(teams['issuekey']))]
info = info[info.columns[0:2]]
info['createdate'] =pd.to_datetime(info['createdate'])
info.sort_values(by='createdate',inplace=True)
info = info[info['issuekey'].isin(teams['issuekey'])]
# print(info['createdate'].min(),info['createdate'].max())
split = int(info.shape[0]*8/10)
# info = info.drop(columns=['createdate'])
train = info.iloc[:split]
test = info.iloc[split:]
# print(train['createdate'].min(),train['createdate'].max())
# print(test['createdate'].min(),test['createdate'].max())
trainingteam =  teams[(teams['issuekey'].isin(train['issuekey']))]
testingteam =  teams[(teams['issuekey'].isin(test['issuekey']))]

In [ ]:
trainingdev = set(trainingteam['dev'])
trainingint = set(trainingteam['integrator'])
trainingpeer = set(trainingteam['peer'])
trainingtester = set(trainingteam['tester'])
trainingassi = set(assignees[(assignees['issuekey'].isin(train['issuekey']))]['username'])
testingdev = set(testingteam['dev'])
testingint = set(testingteam['integrator'])
testingpeer = set(testingteam['peer'])
testingtester = set(testingteam['tester'])
testingassi = set(assignees[(assignees['issuekey'].isin(test['issuekey']))]['username'])
nottestingdev=testingdev-trainingdev
nottestingint=testingint-trainingint
nottestingpeer=testingpeer-trainingpeer
nottestingtester=testingtester-trainingtester
nottestingassi=testingassi-trainingassi

invalidissue = set()

for index, row in tqdm(testingteam.iterrows(),total=testingteam.shape[0]):
    if row['dev'] in nottestingdev or row['integrator'] in nottestingint or row['peer'] in nottestingpeer or row['tester'] in nottestingtester or assignees[(assignees['issuekey']==row['issuekey'])].iloc[0]['username'] in nottestingassi:
        invalidissue.add(row['issuekey'])
testingteam = testingteam[(~testingteam['issuekey'].isin(invalidissue))]
test= pd.DataFrame({'issuekey':testingteam['issuekey'].unique()})


In [ ]:
testingpp = set(testingteam['dev'])
testingpp.update(testingteam['integrator'])
testingpp.update(testingteam['peer'])
testingpp.update(testingteam['tester'])

In [ ]:
# train.to_csv(filepathhelper.path(dataset,'trainissuekey.csv'))
# test.to_csv(filepathhelper.path(dataset,'testissuekey.csv'))

In [ ]:
trainingteam = trainingteam[(trainingteam['issuekey'].isin(winissues['issuekey']))]
testingteam = testingteam[(testingteam['issuekey'].isin(winissues['issuekey']))]

In [ ]:
with open(filepathhelper.path(dataset,'ItoC'),'rb') as f:
    ItoC = pickle.load(f)

In [ ]:
assignees.set_index('issuekey',inplace=True)


# testing dataset

In [ ]:
issue_team = {}
for key,issue in tqdm(testingteam.iterrows(),total=testingteam.shape[0]):
#     print(str(issue['dev'])=='nan')
    issuekey = issue['issuekey']
    if issuekey not in issue_team:
        issue_team[issuekey]={'developer':set(),'integrator':set(),'tester':set(),'reviewer':set(),'assignee':set()}
    team = issue_team[issuekey]
    if str(issue['dev'])!='nan':
        team['developer'].add(issue['dev'])
    if str(issue['integrator'])!='nan':
        team['integrator'].add(issue['integrator'])
    if str(issue['peer'])!='nan':
        team['reviewer'].add(issue['peer'])
    if str(issue['tester'])!='nan':
        team['tester'].add(issue['tester'])
    asn = assignees.loc[issuekey]
    if asn is not None:
        nam = asn['username']
        if str(nam)!='nan':
            team['assignee'].add(nam)

In [ ]:
issuecomponent = pd.read_csv(filepathhelper.path(dataset,'component_title.csv'),sep=';;;',engine='python')
ItoC = {}
for key,issue in tqdm(issuecomponent.iterrows(),total=issuecomponent.shape[0]):
    issuekey = issue['issuekey']
    component = issue['component']
    if issuekey not in ItoC:
        ItoC[issuekey] = set()
    ItoC[issuekey].add(component)


In [ ]:
rolemap = {'developer':'dev','tester':'tester','reviewer':'peer','integrator':'integrator'}
inputs = []
for issuekey in tqdm(issue_team):    
    roles = issue_team[issuekey]
#     context = descript.loc[issuekey]['description']
    context=''
    if issuekey in ItoC:
        components = list(ItoC[issuekey])
    else:
        components = []
    inp = {'issuekey':issuekey,'context':context,'components':components,'assignee': list(roles['assignee'])[0]}
    team = []
    for role in roles:
        if role != 'assignee':
            for i in range(1,len(roles[role])+1):
                team.append(rolemap[role]+str(i))
    inp['team']=team
    inputs.append(inp)

In [ ]:
with open(filepathhelper.path(dataset,'input_test.json'), 'w') as outfile:
    json.dump(inputs, outfile)